In [26]:
import datetime
import gc
import joblib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import scipy.stats
import seaborn as sns
import sys
sys.path.append("../../")
import time
import warnings
warnings.simplefilter("ignore")
from collections import defaultdict
from itertools import repeat, combinations
from pandarallel import pandarallel
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tqdm import tqdm

In [27]:
from utils.common import (
    sigmoid, pad_column_name
)
from utils.constants import *
from utils.eval_helpers import (
    plot_roc_curves, plot_feature_importance, 
    amex_metric, get_final_metric_df, amex_metric_np, lgb_amex_metric
)
from utils.eda_helpers import (
    plot_missing_proportion_barchart, plot_heatmap,
    get_cols, insert_row_number, plot_train_test_distribution
)
from utils.extraction_helpers import read_file
from utils.feature_group import CATEGORY_COLUMNS
from utils.preprocess_helpers import clip_col

In [28]:
from feature_engineering_helpers import feature_gen_pipeline, clip_all, round_all, convert_all

In [29]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
pandarallel.initialize(nb_workers=16, progress_bar=False, use_memory_fs=False)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [31]:
START = time.time()

### Read Data

In [10]:
labels = read_file(f"../{RAW_DATA_PATH}/train_labels.csv")

Shape of data: (458913, 2)


In [11]:
%%time
train = read_file(f"../{INTERIM_DATA_PATH}/v5/train_parquet/train_all_variables.parquet")
test1 = read_file(f"../{INTERIM_DATA_PATH}/v5/test_parquet/test_all_variables1.parquet")
test2 = read_file(f"../{INTERIM_DATA_PATH}/v5/test_parquet/test_all_variables2.parquet")

Shape of data: (5531451, 189)
Shape of data: (5681079, 188)
Shape of data: (5682683, 188)
CPU times: user 54.5 s, sys: 1min 9s, total: 2min 4s
Wall time: 1min 4s


In [12]:
%%time
test = pd.concat([test1, test2], ignore_index=True)
del test1, test2

CPU times: user 11.3 s, sys: 33.1 s, total: 44.4 s
Wall time: 59 s


In [32]:
gc.collect()

12520

In [14]:
df_list = [train, test]

In [15]:
# %%time
# n = train.nunique()
# binary_features = n[n == 2].index.tolist()

### Transform Train Set

In [33]:
%%time
train_agg, keep_column = feature_gen_pipeline(train)

Done insertion
Average done
Minimum done
Maximum done
Standard Deviation done
Last entry done
First entry done
Second last entry done
Third last entry done
MA2 for Recency 1 done
MA2 for Recency 2 done
MA2 for Recency 3 done
MA3 for Recency 1 done
MA3 for Recency 2 done
MA3 for least Recency done


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 173/173 [01:40<00:00,  1.72it/s]

CPU times: user 2min 38s, sys: 1min 21s, total: 3min 59s
Wall time: 4min 22s


In [34]:
train_agg["target"] = labels["target"].values
print(f"Size: {sys.getsizeof(train_agg) / 1e9} GB, Shape: {train_agg.shape}")

Size: 14.388695549 GB, Shape: (458913, 5063)


In [35]:
train_agg = convert_all(train_agg)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2767/2767 [02:14<00:00, 20.57it/s]


In [36]:
train_agg = clip_all(train_agg)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 107/107 [00:03<00:00, 30.12it/s]


In [37]:
train_agg = round_all(train_agg)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5019/5019 [00:39<00:00, 125.59it/s]


In [38]:
print(f"Size: {sys.getsizeof(train_agg) / 1e9} GB, Shape: {train_agg.shape}")

Size: 9.375529937 GB, Shape: (458913, 5063)


In [25]:
train_agg.shape

(458913, 5063)

In [40]:
# train_agg.to_parquet(f"./train_agg.parquet")

In [44]:
# del train_agg, train

### Transform Test

In [45]:
%%time
test_agg, keep_column = feature_gen_pipeline(test)

Done insertion
Average done
Minimum done
Maximum done
Standard Deviation done
Last entry done
First entry done
Second last entry done
Third last entry done
MA2 for Recency 1 done
MA2 for Recency 2 done
MA2 for Recency 3 done
MA3 for Recency 1 done
MA3 for Recency 2 done
MA3 for least Recency done


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 173/173 [02:13<00:00,  1.30it/s]

CPU times: user 5min 10s, sys: 7min 2s, total: 12min 13s
Wall time: 14min 55s


In [46]:
print(f"Size: {sys.getsizeof(test_agg) / 1e9} GB, Shape: {test_agg.shape}")

Size: 29.004975209 GB, Shape: (924621, 5062)


In [47]:
test_agg = convert_all(test_agg)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2773/2773 [05:13<00:00,  8.86it/s]


In [48]:
test_agg = clip_all(test_agg)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 108/108 [00:08<00:00, 13.30it/s]


In [49]:
test_agg = round_all(test_agg)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5018/5018 [02:27<00:00, 33.96it/s]


In [52]:
print(f"Size: {sys.getsizeof(test_agg) / 1e9} GB, Shape: {test_agg.shape}")

Size: 18.889621469 GB, Shape: (924621, 5062)


In [53]:
test_agg.shape

(924621, 5062)

In [54]:
test_agg.to_parquet(f"./test_agg.parquet")

In [55]:
del test_agg, test